# Finetuning a LLM for a new task using qlora and custom heads.
In the *text_classification_linear_probe* notebook we found that the output of the third to last block of a transformer trained on causal_lm has the most linearly seperable information on the sentiment of a sentence. To get even better performance on the sentiment prediction task, we can finetune the whole transformer instead of only the newly added head. In this example, we will jointly train a qlora on the body of the transformer while also fully training a newly initialized head for sentiment classification.

**GPU Requirements:** For running with GPT-2 you may be fine with just 8GB of GPU RAM. With about 24GB you should be able to run any 7B or 13B model. With 80GB (A100) GPU you may be able to run a 70B model.

In [1]:
from transformer_heads import create_headed_qlora
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    MistralForCausalLM,
    Trainer,
    BitsAndBytesConfig,
    TrainingArguments,
    GPT2Model,
    GPT2LMHeadModel,
)
from transformer_heads.util.helpers import DataCollatorWithPadding, get_model_params
from peft import LoraConfig
from transformer_heads.config import HeadConfig
from transformer_heads.util.model import print_trainable_parameters
from transformer_heads.util.evaluate import (
    evaluate_head_wise,
    get_top_n_preds,
    get_some_preds,
)
import torch
import pandas as pd

In [2]:
# GPT2 is the fastest and requires fewest memory. However, this works just the same with any Llama or Mistral model. Just change model_path to its huggingface path.
model_path = "gpt2"
train_epochs = 1
eval_epochs = 1
logging_steps = 100
train_batch_size = 4
eval_batch_size = 4

In [3]:
# Parameters
model_path = "gpt2"
train_epochs = 1
eval_epochs = 1
logging_steps = 100

In [4]:
model_params = get_model_params(model_path)
model_class = model_params["model_class"]
hidden_size = model_params["hidden_size"]
vocab_size = model_params["vocab_size"]
print(model_params)

{'vocab_size': 50257, 'n_positions': 1024, 'n_embd': 768, 'n_layer': 12, 'n_head': 12, 'n_inner': None, 'activation_function': 'gelu_new', 'resid_pdrop': 0.1, 'embd_pdrop': 0.1, 'attn_pdrop': 0.1, 'layer_norm_epsilon': 1e-05, 'initializer_range': 0.02, 'summary_type': 'cls_index', 'summary_use_proj': True, 'summary_activation': None, 'summary_first_dropout': 0.1, 'summary_proj_to_labels': True, 'scale_attn_weights': True, 'use_cache': True, 'scale_attn_by_inverse_layer_idx': False, 'reorder_and_upcast_attn': False, 'bos_token_id': 50256, 'eos_token_id': 50256, 'return_dict': True, 'output_hidden_states': False, 'output_attentions': False, 'torchscript': False, 'torch_dtype': None, 'use_bfloat16': False, 'tf_legacy_loss': False, 'pruned_heads': {}, 'tie_word_embeddings': True, 'chunk_size_feed_forward': 0, 'is_encoder_decoder': False, 'is_decoder': False, 'cross_attention_hidden_size': None, 'add_cross_attention': False, 'tie_encoder_decoder': False, 'max_length': 20, 'min_length': 0, '

Only one head for now, as we aim to maximize the performance of the head hooked at layer -3 (third to last).

In [5]:
head_configs = [
    HeadConfig(
        name=f"imdb_head_3",
        layer_hook=-3,
        in_size=hidden_size,
        output_activation="linear",
        pred_for_sequence=True,
        loss_fct="cross_entropy",
        num_outputs=2,
    )
]

In [6]:
dd = load_dataset("imdb")

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token


def tokenize_function(examples):
    out = tokenizer(examples["text"], padding=False, truncation=True)
    for hc in head_configs:
        out[hc.name] = examples["label"]
    return out


for split in dd.keys():
    dd[split] = dd[split].filter(function=lambda example: len(example["text"]) > 10)
    dd[split] = dd[split].shuffle()
    dd[split] = dd[split].map(tokenize_function, batched=True)

dd.set_format(
    type="torch",
    columns=["input_ids", "attention_mask"] + [x.name for x in head_configs],
)
for split in dd.keys():
    dd[split] = dd[split].remove_columns(["text", "label"])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

We want to train a LoRA, so we have to define a LoRA config. The *create_headed_qlora* function will automatically set up our model with *requires_grad* True only for the LoRA parameters and for the parameters of the heads.

Setting *target_modules=None* in the qlora config will make *create_headed_qlora* create LoRA modules for all linear layers in the transformer.

In [8]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_in_8bit=False,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_compute_dtype=torch.float32,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)
lora_config = LoraConfig(
    r=32,
    lora_alpha=16,
    target_modules=None,
    lora_dropout=0.0,
    bias="none",
    task_type="CAUSAL_LM",
)
model = create_headed_qlora(
    base_model_class=model_class,
    model_name=model_path,
    quantization_config=quantization_config,
    lora_config=lora_config,
    head_configs=head_configs,
    fully_trained_heads=True,
    device_map={"": torch.cuda.current_device()},
)

Some weights of TransformerWithHeads were not initialized from the model checkpoint at gpt2 and are newly initialized: ['heads.imdb_head_3.lins.0.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
print_trainable_parameters(model)

all params: 86692608 || trainable params: 4720128 || trainable%: 5.44467182253878
params by dtype: defaultdict(<class 'int'>, {torch.float32: 44225280, torch.uint8: 42467328})
trainable params by dtype: defaultdict(<class 'int'>, {torch.float32: 4720128})


In [10]:
collator = DataCollatorWithPadding(
    feature_name_to_padding_value={
        "input_ids": tokenizer.pad_token_id,
        "attention_mask": 0,
    }
)

In [11]:
args = TrainingArguments(
    output_dir="imdb_linear_probe",
    learning_rate=0.0002,
    num_train_epochs=train_epochs,
    logging_steps=logging_steps,
    do_eval=False,
    remove_unused_columns=False,
    optim="paged_adamw_32bit",
    gradient_checkpointing=True,
    lr_scheduler_type="constant",
    ddp_find_unused_parameters=False,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
)
trainer = Trainer(
    model,
    args=args,
    train_dataset=dd["train"],
    data_collator=collator,
)
trainer.train()

wandb: Currently logged in as: ykeller (chm-hci). Use `wandb login --relogin` to force relogin


wandb: wandb version 0.16.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: Tracking run with wandb version 0.16.3


wandb: Run data is saved locally in /raven/u/ykeller/transformer_heads/wandb/run-20240324_012441-lvqa762s
wandb: Run `wandb offline` to turn off syncing.


wandb: Syncing run wobbly-grass-214


...

Checkpoint destination directory imdb_linear_probe/checkpoint-5500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


/u/ykeller/conda-envs/sh_finetuning/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/u/ykeller/conda-envs/sh_finetuning/lib/python3.10/site-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Checkpoint destination directory imdb_linear_probe/checkpoint-6000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


/u/ykeller/conda-envs/sh_finetuning/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/u/ykeller/conda-envs/sh_finetuning/lib/python3.10/site-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


TrainOutput(global_step=6250, training_loss=0.6409403942871094, metrics={'train_runtime': 393.6129, 'train_samples_per_second': 63.514, 'train_steps_per_second': 15.879, 'total_flos': 7122221053968384.0, 'train_loss': 0.6409403942871094, 'epoch': 1.0})

In [12]:
print(
    evaluate_head_wise(
        model, dd["test"], collator, epochs=eval_epochs, batch_size=eval_batch_size
    )
)


Evaluating:   0%|          | 0/6250 [00:00<?, ?it/s]


Evaluating:   0%|          | 3/6250 [00:00<04:35, 22.69it/s]


Evaluating:   0%|          | 6/6250 [00:00<04:05, 25.47it/s]


Evaluating:   0%|          | 9/6250 [00:00<04:51, 21.41it/s]


Evaluating:   0%|          | 12/6250 [00:00<05:46, 17.99it/s]

...


Evaluating: 100%|█████████▉| 6243/6250 [05:24<00:00, 20.53it/s]


Evaluating: 100%|█████████▉| 6246/6250 [05:24<00:00, 20.99it/s]


Evaluating: 100%|█████████▉| 6249/6250 [05:24<00:00, 22.70it/s]


Evaluating: 100%|██████████| 6250/6250 [05:24<00:00, 19.24it/s]

(0.5582045646527409, {'imdb_head_3': 0.5582045646527409})


In [13]:
ins, preds, ground_truths = get_some_preds(
    model, dd["test"], tokenizer, n=20, classification=True
)
print(
    pd.DataFrame(
        list(zip(ins, preds["imdb_head_5"], ground_truths["imdb_head_5"])),
        columns=["review", "label", "ground_truth"],
    )
)


Predicting:   0%|          | 0/20 [00:00<?, ?it/s]


Predicting:  20%|██        | 4/20 [00:00<00:00, 31.04it/s]


Predicting:  40%|████      | 8/20 [00:00<00:00, 32.54it/s]


Predicting:  60%|██████    | 12/20 [00:00<00:00, 33.05it/s]


Predicting:  80%|████████  | 16/20 [00:00<00:00, 33.10it/s]


Predicting: 100%|██████████| 20/20 [00:00<00:00, 33.22it/s]


Predicting: 100%|██████████| 20/20 [00:00<00:00, 31.39it/s]

Empty DataFrame
Columns: [review, label, ground_truth]
Index: []
